# Webcrawling Study
* 참고: [나만의 웹 크롤러러 만들기 시리즈](https://beomi.github.io/gb-crawling/)
* 타겟: [어린이동아](http://kids.donga.com)

## Requests

In [314]:
# !pip install requests
import requests

# HTTP GET Request
## 어린이 동아 뉴스 리스트
req = requests.get("http://kids.donga.com/?ptype=article&psub=news")

html = req.text           # HTML (use this source!)
header = req.headers      # header
status = req.status_code  # status
is_ok = req.ok            # status boolean

## BeautifulSoup

In [19]:
# !pip install bs4
from bs4 import BeautifulSoup

# convert HTML source to pythob object
## BeautifulSoup({html}, {parser})
soup = BeautifulSoup(html, "html.parser")

**CSS Selector**

* **full selector**: 
body > div.wrap_all > div.content > div.page_area > div.article_list > ul > li:nth-child(1) > dl > dt > a

* **target selector**: 
dt > a

In [26]:
# CSS Selector
article_titles = soup.select("dt > a")
article_titles

[<a href="https://kids.donga.com?ptype=article&amp;no=20220202124700920857&amp;psub=news&amp;gbn=01">2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은?...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123848709079&amp;psub=news&amp;gbn=01">[뉴스쏙 시사쑥] 영국에서 SNS 사진 ‘보정 유무 표기’ 법안...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123358514117&amp;psub=news&amp;gbn=01">[뉴스 브리핑] 일본, ‘사도광산’ 세계유산 추천 결정</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120934178503&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 한국 축구, 10회 연속 월드컵 본선 진출 위...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120612605158&amp;psub=news&amp;gbn=01">[월드 뉴스] 미얀마 쿠데타 1년… 여전히 이어지는 시위</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120507717054&amp;psub=news&amp;gbn=01">[한자뉴스] 하늘을 나는 요트… 약 7465억 원</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220127140638502004&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 낙서한다고 혼났던 영국 소년, 나이키 디자이너...</a>,
 <a href="https://

In [57]:
# selector elements
print("text: ", article_titles[0].text)
print("href: ", article_titles[0].get("href"))

text:  2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은?...
href:  https://kids.donga.com?ptype=article&no=20220202124700920857&psub=news&gbn=01


**Sub Crawling**

* **Writer**: 
body > div.wrap_all > div.content > div.page_area > div.at_cont > div.at_cont_title > div.at_info > **ul > li.writer**

* **Upload Date**: 
body > div.wrap_all > div.content > div.page_area > div.at_cont > div.at_cont_title > div.at_info > **ul > li.upload_date**

* **Contents**:
body > div.wrap_all > div.content > div.page_area > div.at_cont > **div.cont_view** > div > p:nth-child(4) > span

In [257]:
# sub crawling
href = article_titles[0].get("href")
sub_req = requests.get(href)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, "html.parser")

In [197]:
# Writer
## writer = sub_soup.select_one("li.writer").text
writer = "\r\n\t\t\t\t\t홍길동 기자\t\t\t\t\t"
writer

'\r\n\t\t\t\t\t홍길동 기자\t\t\t\t\t'

In [198]:
## writer regex
import re
writer = re.sub("\W", "", writer.split(" ")[0])
writer

'홍길동'

In [199]:
# Upload Date
upload_date = sub_soup.select_one("li.upload_date").text
upload_date

'2022-02-02 12:47:00'

In [258]:
# Upload Date
content = sub_soup.select_one("div.at_content").text
content

"\n스키점프대 옆에 선수의 기록을 알리는 오메가의 전광판이 설치된 모습. 오메가 홈페이지 캡처지난해 2020 도쿄 올림픽에서는 사상 최초로 육상 선수의 몸에 모션 센서(동작감지기)를 부착해 실시간 위치, 속도, 가속도(점점 더해지는 속도), 감속도(점점 줄어드는 속도)를 측정했다. 모션 센서를 개발한 것은 1932년부터 올림픽의 타임키퍼(시간 기록원)를 맡고 있는 스위스의 시계 브랜드인 ‘오메가’다.올림픽 경기 중계를 유심히 보다 보면 곳곳에서 ‘OMEGA(Ω)’라는 글자와 로고를 찾아볼 수 있다. 0.001초 차이로 승부가 결정되곤 하는 올림픽의 세계에서 오메가는 90년간 정밀한 시계 기술로 선수들의 기록을 측정해왔다. 이제 오메가는 단순히 기록을 재는 것을 넘어 경기력까지 분석하는 스마트 시스템을 갖췄다. 2022 베이징 올림픽에서도 타임키퍼로 나서는 오메가는 어떤 기술을 선보일까.피겨 선수 점프 높이 측정 ‘OK’오메가가 2022 베이징 겨울올림픽 피겨스케이팅 경기장에 카메라를 설치한 모습을 그린 그림고난도의 점프 기술과 섬세한 연기력을 모두 갖춰야 높은 점수를 받는 스포츠인 피겨스케이팅은 겨울올림픽의 ‘꽃’으로 불린다.베이징 올림픽에서는 피겨스케이팅 선수들의 점프 기술을 낱낱이 뜯어 볼 수 있게 됐다. 오메가는 선수들의 움직임과 위치를 파악하는 ‘모션 센서 및 포지셔닝 감지 시스템’을 피겨스케이팅에도 도입한다고 최근 밝혔다. 2018 평창 겨울올림픽에서 공식적으로 처음 도입된 모션 센서 및 포지셔닝 감지 시스템은 선수의 몸이나 운동기기 등에 부착해 움직임을 데이터화하는 기술을 말한다.이와 더불어 오메가는 빙상 경기장 주변에 설치한 6대의 카메라로 피겨스케이팅 싱글 경기 중 선수들의 점프 높이와 비거리, 체공(공중에 머물러 있음) 시간 같은 데이터를 실시간으로 측정한다. 데이터를 바탕으로 선수들은 자신의 경기방식을 다른 선수와 비교·분석할 수 있게 되고, 시청자들도 선수들의 동작을 자세히 이해하게 될 것으로 보인다.하키에선 특수 디스플레이 도입아이스하키

In [259]:
content = content.split("▶어린이동아")[0]
re.sub("[\xa0\u200b\n]", "", content)

'스키점프대 옆에 선수의 기록을 알리는 오메가의 전광판이 설치된 모습. 오메가 홈페이지 캡처지난해 2020 도쿄 올림픽에서는 사상 최초로 육상 선수의 몸에 모션 센서(동작감지기)를 부착해 실시간 위치, 속도, 가속도(점점 더해지는 속도), 감속도(점점 줄어드는 속도)를 측정했다. 모션 센서를 개발한 것은 1932년부터 올림픽의 타임키퍼(시간 기록원)를 맡고 있는 스위스의 시계 브랜드인 ‘오메가’다.올림픽 경기 중계를 유심히 보다 보면 곳곳에서 ‘OMEGA(Ω)’라는 글자와 로고를 찾아볼 수 있다. 0.001초 차이로 승부가 결정되곤 하는 올림픽의 세계에서 오메가는 90년간 정밀한 시계 기술로 선수들의 기록을 측정해왔다. 이제 오메가는 단순히 기록을 재는 것을 넘어 경기력까지 분석하는 스마트 시스템을 갖췄다. 2022 베이징 올림픽에서도 타임키퍼로 나서는 오메가는 어떤 기술을 선보일까.피겨 선수 점프 높이 측정 ‘OK’오메가가 2022 베이징 겨울올림픽 피겨스케이팅 경기장에 카메라를 설치한 모습을 그린 그림고난도의 점프 기술과 섬세한 연기력을 모두 갖춰야 높은 점수를 받는 스포츠인 피겨스케이팅은 겨울올림픽의 ‘꽃’으로 불린다.베이징 올림픽에서는 피겨스케이팅 선수들의 점프 기술을 낱낱이 뜯어 볼 수 있게 됐다. 오메가는 선수들의 움직임과 위치를 파악하는 ‘모션 센서 및 포지셔닝 감지 시스템’을 피겨스케이팅에도 도입한다고 최근 밝혔다. 2018 평창 겨울올림픽에서 공식적으로 처음 도입된 모션 센서 및 포지셔닝 감지 시스템은 선수의 몸이나 운동기기 등에 부착해 움직임을 데이터화하는 기술을 말한다.이와 더불어 오메가는 빙상 경기장 주변에 설치한 6대의 카메라로 피겨스케이팅 싱글 경기 중 선수들의 점프 높이와 비거리, 체공(공중에 머물러 있음) 시간 같은 데이터를 실시간으로 측정한다. 데이터를 바탕으로 선수들은 자신의 경기방식을 다른 선수와 비교·분석할 수 있게 되고, 시청자들도 선수들의 동작을 자세히 이해하게 될 것으로 보인다.하키에선 특수 디스플레이 도입아이스하키 경

## Full Web Crawler

In [310]:
%%time

import requests
from bs4 import BeautifulSoup
import re

req = requests.get("http://kids.donga.com/?ptype=article&psub=news")
html = req.text
soup = BeautifulSoup(html, "html.parser")

article_titles = soup.select("dt > a")

data = []
for title in article_titles:
    json_data = {}
    json_data["title"] = title.text
    json_data["href"] = title.get("href")

    sub_req = requests.get(json_data["href"])
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, "html.parser")
    
    writer = sub_soup.select_one("li.writer").text
    json_data["writer"] = re.sub("\W", "", writer.split(" ")[0])

    json_data["upload_date"] = sub_soup.select_one("li.upload_date").text

    content = sub_soup.select_one("div.at_content").text
    content = content.split("▶어린이동아")[0]
    content = re.sub("[\xa0\u200b\n]", "", content)
    json_data["content"] = content
    
    data.append(json_data)

CPU times: user 663 ms, sys: 17.4 ms, total: 680 ms
Wall time: 10.3 s


In [311]:
data

[{'content': '스키점프대 옆에 선수의 기록을 알리는 오메가의 전광판이 설치된 모습. 오메가 홈페이지 캡처지난해 2020 도쿄 올림픽에서는 사상 최초로 육상 선수의 몸에 모션 센서(동작감지기)를 부착해 실시간 위치, 속도, 가속도(점점 더해지는 속도), 감속도(점점 줄어드는 속도)를 측정했다. 모션 센서를 개발한 것은 1932년부터 올림픽의 타임키퍼(시간 기록원)를 맡고 있는 스위스의 시계 브랜드인 ‘오메가’다.올림픽 경기 중계를 유심히 보다 보면 곳곳에서 ‘OMEGA(Ω)’라는 글자와 로고를 찾아볼 수 있다. 0.001초 차이로 승부가 결정되곤 하는 올림픽의 세계에서 오메가는 90년간 정밀한 시계 기술로 선수들의 기록을 측정해왔다. 이제 오메가는 단순히 기록을 재는 것을 넘어 경기력까지 분석하는 스마트 시스템을 갖췄다. 2022 베이징 올림픽에서도 타임키퍼로 나서는 오메가는 어떤 기술을 선보일까.피겨 선수 점프 높이 측정 ‘OK’오메가가 2022 베이징 겨울올림픽 피겨스케이팅 경기장에 카메라를 설치한 모습을 그린 그림고난도의 점프 기술과 섬세한 연기력을 모두 갖춰야 높은 점수를 받는 스포츠인 피겨스케이팅은 겨울올림픽의 ‘꽃’으로 불린다.베이징 올림픽에서는 피겨스케이팅 선수들의 점프 기술을 낱낱이 뜯어 볼 수 있게 됐다. 오메가는 선수들의 움직임과 위치를 파악하는 ‘모션 센서 및 포지셔닝 감지 시스템’을 피겨스케이팅에도 도입한다고 최근 밝혔다. 2018 평창 겨울올림픽에서 공식적으로 처음 도입된 모션 센서 및 포지셔닝 감지 시스템은 선수의 몸이나 운동기기 등에 부착해 움직임을 데이터화하는 기술을 말한다.이와 더불어 오메가는 빙상 경기장 주변에 설치한 6대의 카메라로 피겨스케이팅 싱글 경기 중 선수들의 점프 높이와 비거리, 체공(공중에 머물러 있음) 시간 같은 데이터를 실시간으로 측정한다. 데이터를 바탕으로 선수들은 자신의 경기방식을 다른 선수와 비교·분석할 수 있게 되고, 시청자들도 선수들의 동작을 자세히 이해하게 될 것으로 보인다.하키에선 특수 디스

In [313]:
data[3]["content"]

'월드컵 10회 연속 진출을 확정한 우리나라 대표팀 선수들과 코칭 스태프가 기념촬영을 하고 있다. 두바이=뉴시스1일 열린 시리아와의 경기에서 추가골을 터뜨린 권창훈(맨 왼쪽)이 엄지를 치켜세우며 동료들과 기뻐하고 있다우리나라 축구가 10회 연속 월드컵 본선 진출이라는 위업(위대한 업적)을 달성했다. 올해로 22회째를 맞는 월드컵에서 10회 연속으로 월드컵 본선 진출을 이룬 나라는 우리나라를 포함해 세계에서 6개 나라뿐이다.파울루 벤투(포르투갈) 감독이 이끄는 우리나라 축구대표팀은 1일(현지시간) 아랍에미리트(UAE) 두바이의 라시드 스타디움에서 열린 2022 국제축구연맹(FIFA) 카타르 월드컵 아시아 지역 최종예선 8차전 시리아와의 경기에서 김진수(30·전북 현대)와 권창훈(28·김천 상무)의 연속골에 힘입어 2대0으로 승리했다. 이로써 최종예선에서 패배 없이 6승 2무를 기록한 우리나라는 남은 예선 2경기 결과에 상관없이 월드컵 본선 진출을 확정지었다. 2개조로 나뉘어 치러지는 아시아 지역 최종예선에선 각 조의 1, 2위 국가가 본선에 직행(바로 감)하는데, 우리나라는 현재 A조 2위에 올라있다.우리나라는 1986년 멕시코에서 열린 월드컵 본선 대회부터 시작해 올해 카타르에서 열릴 월드컵까지 10회 연속으로 본선 무대를 밟는다. 월드컵 본선 대회에 10회 연속 이상 진출한 나라는 우리나라를 제외하고 전 세계에서 브라질, 독일, 이탈리아, 아르헨티나, 스페인뿐이다.아울러 우리나라는 카타르 월드컵 본선 진출을 확정한 15번째 국가가 됐다. 개최국 카타르를 비롯해 스페인, 프랑스, 네덜란드, 잉글랜드, 독일, 브라질, 아르헨티나 등이 우리나라보다 앞서 카타르 월드컵 본선 진출을 확정지었다.[한 뼘 더] 11월, 카타르를 달굴 축구 열기!2022 FIFA 카타르 월드컵은 아랍권에서 처음 열리는 월드컵! 올해 11월 21일부터 12월 18일까지 카타르의 수도 도하를 비롯한 5개 도시에서 열려요. 월드컵이 11월에 개막하는 것은 이번이 처음. 과거엔 6월에 열렸지만 

## 해결 과제
1. 이미지 참조 글과 기사가 붙음
2. unicode 공백 문자
3. 날짜 별/페이지 별 크롤링 자동화

body > div.wrap_all > div.content > div.page_area > div.article_list > ul > li:nth-child(5) > dl > dd:nth-child(3)

### 날짜 별/페이지 별 크롤링 자동화

In [348]:
article_titles

[<a href="https://kids.donga.com?ptype=article&amp;no=20220202124700920857&amp;psub=news&amp;gbn=01">2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은?...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123848709079&amp;psub=news&amp;gbn=01">[뉴스쏙 시사쑥] 영국에서 SNS 사진 ‘보정 유무 표기’ 법안...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123358514117&amp;psub=news&amp;gbn=01">[뉴스 브리핑] 일본, ‘사도광산’ 세계유산 추천 결정</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120934178503&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 한국 축구, 10회 연속 월드컵 본선 진출 위...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120612605158&amp;psub=news&amp;gbn=01">[월드 뉴스] 미얀마 쿠데타 1년… 여전히 이어지는 시위</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120507717054&amp;psub=news&amp;gbn=01">[한자뉴스] 하늘을 나는 요트… 약 7465억 원</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220127140638502004&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 낙서한다고 혼났던 영국 소년, 나이키 디자이너...</a>,
 <a href="https://

In [356]:
from datetime import datetime

today = datetime.today().strftime("%Y%m%d")
today

'20220202'

In [366]:
today_article = [i for i in article_titles if today in i.get("href")]
today_article

[<a href="https://kids.donga.com?ptype=article&amp;no=20220202124700920857&amp;psub=news&amp;gbn=01">2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은?...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123848709079&amp;psub=news&amp;gbn=01">[뉴스쏙 시사쑥] 영국에서 SNS 사진 ‘보정 유무 표기’ 법안...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123358514117&amp;psub=news&amp;gbn=01">[뉴스 브리핑] 일본, ‘사도광산’ 세계유산 추천 결정</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120934178503&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 한국 축구, 10회 연속 월드컵 본선 진출 위...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120612605158&amp;psub=news&amp;gbn=01">[월드 뉴스] 미얀마 쿠데타 1년… 여전히 이어지는 시위</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120507717054&amp;psub=news&amp;gbn=01">[한자뉴스] 하늘을 나는 요트… 약 7465억 원</a>]